In [ ]:
import pandas as pd
import numpy as np
import re
import string
from collections import Counter
from sklearn.model_selection import train_test_split
import gc

In [ ]:
import nltk
from nltk.corpus import stopwords

In [ ]:
from  tensorflow.keras.preprocessing.text import Tokenizer,one_hot
from  tensorflow.keras.preprocessing.sequence import pad_sequences
from  tensorflow.keras.models import Sequential
from  tensorflow.keras.layers import Dense,Dropout,Flatten,Conv1D,MaxPooling1D
from  tensorflow.keras.layers import Embedding


In [ ]:
from tensorflow.keras.layers import LeakyReLU,PReLU,ELU

In [ ]:
import tensorflow as tf

In [ ]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')]

In [ ]:
df = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
sub = pd.read_csv('SampleSubmission.csv')

In [ ]:
df = df[df.label != 0]

In [ ]:
tot_data = pd.concat([df,test])

In [ ]:
sub.head()

,ID,label
0,2DDHQW9,0
1,5HY6UEY,0
2,ATNVUJX,0
3,Q9XYVOQ,0
4,TOAHLRH,0


In [ ]:
test_docs = test.text.values

In [ ]:
test.shape

(30000, 2)

In [ ]:
df.head()

,ID,text,label
0,13P0QT0,3sbaaaaaaaaaaaaaaaaaaaa lek ou le seim riahi o...,-1
1,SKCLXCJ,cha3eb fey9elkoum menghir ta7ayoul ou kressi,-1
2,V1TVXIJ,bereau degage nathef ya slim walahi ya7chiw fi...,-1
3,U0TTYY8,ak slouma,1
4,68DX797,entom titmanou lina a7na 3iid moubarik a7na ch...,-1


In [ ]:
X = df.text.values
y =  df.label.values
X_train_docs, X_test_docs, y_train, y_test = train_test_split(X, y, test_size=0.001,stratify = y,random_state=42)

In [ ]:
X_train_docs.shape

(67466,)

In [ ]:
def clean_not_alpha_numeric(token):
    if not token.isalpha():
        token = token[:4]
        return token
    else:
        return token

In [ ]:
def clean_doc(text):
    tokens = text.split()
    re_punct = re.compile('[%s]' % re.escape(string.punctuation))
    tokens = [re_punct.sub('',word) for word in tokens]
    tokens = [words for words in tokens if len(words) > 1]
    stop_words = stopwords.words('french')
    #tokens = [word for word in tokens if not word in stop_words]
    tokens = [clean_not_alpha_numeric(word) for word in tokens]
    #tokens = [word for word in tokens if word.isalpha()]
    
    return tokens

In [ ]:
def check_isalpha(text):
    token = [token for token in text.split() if not token.isalpha()]
    token = [clean_not_alpha_numeric(word) for word in token]
    return token

In [ ]:
def process_data(array,vocab):
    for i in range(len(array)):
        text = array[i]
        tokens = clean_doc(text)
        vocab.update(tokens)

In [ ]:
vocab = Counter()
text_data = tot_data.text.values
process_data(text_data,vocab)

In [ ]:
print(vocab.most_common(100))

[('ya', 12730), ('fi', 12687), ('el', 11054), ('slim', 9583), ('rabi', 8992), ('si', 5007), ('ca', 4371), ('kol', 4350), ('bech', 4328), ('m3ak', 4199), ('bravo', 4025), ('ma', 3992), ('ken', 3816), ('3la', 3788), ('la', 3705), ('allah', 3560), ('ou', 3195), ('men', 3113), ('slouma', 3086), ('il', 3005), ('eli', 2806), ('m3al', 2618), ('le', 2548), ('ama', 2533), ('3ala', 2497), ('rabbi', 2495), ('jam3', 2303), ('enti', 2208), ('3and', 2200), ('barcha', 2177), ('et', 2176), ('ki', 2076), ('min', 2039), ('3lik', 1983), ('dima', 1950), ('tounes', 1948), ('fih', 1925), ('tawa', 1923), ('mabrouk', 1921), ('cha3', 1915), ('elli', 1853), ('wa', 1808), ('li', 1779), ('les', 1770), ('fil', 1750), ('chay', 1739), ('mouch', 1724), ('we', 1719), ('ena', 1709), ('de', 1700), ('mta3', 1690), ('ti', 1685), ('riahi', 1677), ('club', 1604), ('ye', 1572), ('mala', 1553), ('ana', 1537), ('brabi', 1524), ('3am', 1505), ('fel', 1482), ('m3a', 1467), ('taw', 1453), ('ili', 1378), ('chkoun', 1367), ('3lih',

In [ ]:
min_occurence = 2

tokens = [k for k,c in vocab.items() if c >=min_occurence]

In [ ]:
print(len(tokens))

45861


tokens

In [ ]:
def save_list(lines,filename):
    data = '\n'.join(lines)
    file = open(filename,'w',encoding="utf-8")
    file.write(data)
    file.close()

In [ ]:
import io

def save_list(lines,filename):
    data = '\n'.join(lines)
    with io.open(filename,'w', encoding="utf-8") as file:
        file.write(data)
    #file.close()

save_list(tokens,'vocab.txt')

In [ ]:
def load_doc(filename):
    file  = open(filename,'r')
    text = file.read()
    file.close()
    return text

In [ ]:
vocab =  tokens

vocab = vocab.split()

In [ ]:
len(vocab)

45861

In [ ]:
def process_data_train_test(array,vocab):
    total_array = list()
    for i in range(len(array)):
        text = array[i]
        tokens = clean_doc(text)
        tokens =  [word for word in tokens if word in vocab]
        total_array.append(tokens)
    return total_array

In [ ]:
X_train_docs = process_data_train_test(X_train_docs,vocab)
X_test_docs = process_data_train_test(X_test_docs,vocab)


In [ ]:
test_docs = process_data_train_test(test_docs,vocab)

In [ ]:
def creat_tokenizer(train_docs):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(train_docs)
    return tokenizer

In [ ]:
tot_docs = list()
tot_docs.extend(X_train_docs)
tot_docs.extend(X_test_docs)
tot_docs.extend(test_docs)

In [ ]:
len(tot_docs)

97534

In [ ]:
max_length = max([len(s) for s in X_train_docs[:]])
print('Maximum length: %d' % max_length)

Maximum length: 1345


In [ ]:
max_length = max([len(s) for s in tot_docs])
print('Maximum length: %d' % max_length)

Maximum length: 1345


In [ ]:
def encode_docs(tokenizer, max_length, docs):
    # integer encode
    encoded = tokenizer.texts_to_sequences(docs)
    # pad sequences
    padded = pad_sequences(encoded, maxlen=max_length, padding='post')
    return padded

In [ ]:
# Create Tokenizer
tokenizer = creat_tokenizer(X_train_docs)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1 
print(vocab_size)

45859


In [ ]:
X_train = encode_docs(tokenizer, max_length, X_train_docs)
X_test = encode_docs(tokenizer, max_length, X_test_docs)
test_data = encode_docs(tokenizer, max_length, test_docs)

In [ ]:
def define_model(vocab_size, max_length):
    model = Sequential()
    model.add(Embedding(vocab_size, 100, input_length=max_length))
    model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(10, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    # compile network
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # summarize defined model
    model.summary()
    return model

In [ ]:
def define_model(vocab_size, max_length):
    model = Sequential()
    model.add(Embedding(vocab_size, 100, input_length=max_length))
    model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(10, activation='relu'))
    model.add(Dense(3, activation='sigmoid'))
    # compile network
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # summarize defined model
    model.summary()
    return model

del X_train,X_test,test_data
gc.collect()

del X_train_docs,X_test_docs,test_docs
gc.collect()

In [ ]:
gc.collect()

1812

# `Building the model`

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
import random
import tensorflow as tf
import os
import numpy as np
def fix_randomness(seed_value = 42):
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['PYTHONHASHSEED']=str(seed_value)
    random.seed(seed_value)
    np.random.seed(seed_value)
    tf.random.set_seed(seed_value)

In [ ]:
len(y_train)

60780

In [ ]:
one_hot = OneHotEncoder(sparse=False)
one_hot.fit(y_train.reshape(-1,1))
y_train_  = one_hot.transform(y_train.reshape(-1,1))
y_test_  = one_hot.transform(y_test.reshape(-1,1))

y_train = pd.DataFrame(y_train).replace({1:0,-1:1}).values
y_test = pd.DataFrame(y_test).replace({1:0,-1:1}).values

In [ ]:
%%time
fix_randomness()
model = define_model(vocab_size, max_length)
model.fit(X_train,y_train_,validation_data = (X_test,y_test_),batch_size =  32,epochs=1,verbose=1)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 1345, 100)         4585900   
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 1338, 32)          25632     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 669, 32)           0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 21408)             0         
_________________________________________________________________
dense_8 (Dense)              (None, 10)                214090    
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 22        
Total params: 4,825,644
Trainable params: 4,825,644
Non-trainable params: 0
____________________________________________

`@@@@@@@@@ loss: 0.5395 - accuracy: 0.7747 - val_loss: 0.4849 - val_accuracy: 0.8009 bs = 8`

`loss: 0.3397 - accuracy: 0.7661 - val_loss: 0.2965 - val_accuracy: 0.8047 === 0.7974 binary 0.05tr statify`

`loss: 0.5399 - accuracy: 0.7746 - val_loss: 0.4818 - val_accuracy: 0.7970 === 300 ep === 0.7986 binary 0.1tr statify`

`loss: 0.5398 - accuracy: 0.7747 - val_loss: 0.4811 - val_accuracy: 0.7973 === 300 ep === 0.7994 binary 0.1tr statify`

`loss: 0.5395 - accuracy: 0.7745 - val_loss: 0.4872 - val_accuracy: 0.8030 === 300 ep === 0.7964666 binary 0.1tr no statify`

`loss: 0.5395 - accuracy: 0.7747 - val_loss: 0.4849 - val_accuracy: 0.8009 === 300 ep === 0.7984 binary 0.001tr`

`loss: 0.5478 - accuracy: 0.7687 - val_loss: 0.4911 - val_accuracy: 0.7970 === 300  freq`

`loss: 0.5556 - accuracy: 0.7751 - val_loss: 0.4931 - val_accuracy: 0.7986  tfidf`

`loss: 0.5427 - accuracy: 0.7747 - val_loss: 0.4830 - val_accuracy: 0.7961  count`

`loss: 0.5562 - accuracy: 0.7752 - val_loss: 0.4961 - val_accuracy: 0.7984 === 500 ep`

`loss: 0.5598 - accuracy: 0.7730 - val_loss: 0.5029 - val_accuracy: 0.7976 === 0.7966`

`loss: 0.5652 - accuracy: 0.7692 - val_loss: 0.5095 - val_accuracy: 0.7943 === 0.79413`

In [ ]:
0.805066 - .80299997 

0.002066029999999941

In [ ]:
0.8047 + 0.002066029999999941

0.8067660299999999

In [ ]:
0.7943 - 0.79413

0.00017000000000000348

In [ ]:
0.7976 - 0.7966

0.0010000000000000009

In [ ]:
0.7976 - 0.0050

0.7926

In [ ]:
0.7963 - 0.0050

0.7913

In [ ]:
 0.7939 - 0.0050

0.7889

In [ ]:
0.7907 - 0.7856


0.005099999999999993

In [ ]:
0.7916 - 0.00509

0.7865099999999999

In [ ]:
 0.7666 - 0.7614

0.005199999999999982

In [ ]:
 0.75770563 -0.752733333333333


0.0049722966666669866

In [ ]:
# evaluate
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print('Test Accuracy: %f' % (acc*100))

Test Accuracy: 83.076698


In [ ]:
# evaluate
loss, acc = model.evaluate(X_test, y_test_, verbose=0)
print('Test Accuracy: %f' % (acc*100))

test_pred =np.argmax(model.predict(test_data), axis=-1)
test_pred

%%time 
test_pred =np.argmax(model.predict(test_data), axis=-1)
test_pred

In [ ]:
gc.collect()

In [ ]:
%%time 
test_pred =model.predict_classes(test_data)
test_pred

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
Wall time: 6.12 s


In [ ]:
df.label.value_counts(normalize = True)

 1    0.566218
-1    0.433782
Name: label, dtype: float64

In [ ]:
test_pred = pd.DataFrame(test_pred).replace({0:1,1:-1}) # {1:0,-1:1}

In [ ]:
test_pred[0].value_counts(normalize = True)

 1    0.514033
-1    0.485967
Name: 0, dtype: float64

In [ ]:
sub.label =  test_pred

In [ ]:
sub.to_csv('baseline.csv',index = False)

In [ ]:
a = pd.DataFrame()

In [ ]:
a['ID'] =  test.ID

In [ ]:
a['label'] =  test_pred

In [ ]:
a

,ID,label
0,2DDHQW9,-1
1,5HY6UEY,-1
2,ATNVUJX,1
3,Q9XYVOQ,-1
4,TOAHLRH,1
...,...,...
29995,NHXTL3R,-1
29996,U1YWB2O,1
29997,O3KYLM0,1
29998,W4C38TY,-1


In [ ]:
a.to_csv('cnn_gpu_pos_neg_only.csv',index = False)

In [ ]:
del model
gc.collect()

2879

In [ ]:
gc.collect()

22

`freq = Test Accuracy: 76.727271`

`count = Test Accuracy: 76.645023 733/733 - 10s - loss: 0.0460 - accuracy: 0.9824 - val_loss: 1.1938 - val_accuracy: 0.7665`

`binary = Test Accuracy: 76.761907 : 733/733 - 11s - loss: 0.0451 - accuracy: 0.9826 - val_loss: 1.1911 - val_accuracy: 0.7676`

`tfidf = Test Accuracy: 76.805192 : 733/733 - 10s - loss: 0.0215 - accuracy: 0.9907 - val_loss: 1.6002 - val_accuracy: 0.7681`

`tfidf = Test Accuracy: 75.324672 : 184/184  - 4s 22ms/step - loss: 0.0652 - accuracy: 0.9734 - val_loss: 1.1676 - val_accuracy: 0.7532`